In [1]:
import re
import os
import clip
import copy
import torch
from torch import nn
from PIL import Image
from typing import Union
import torch.nn.functional as F
from clip.model import CLIP, ModifiedResNet, AttentionPool2d

from models.part_cem import PartCEM
%load_ext autoreload
%autoreload 2

device = torch.device('cpu')

/home/richard/.pyenv/versions/3.12.2/envs/research/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
part_cem = PartCEM()

TypeError: super(type, obj): obj must be an instance or subtype of type

In [5]:
x = torch.randn(8, 3, 224, 224)
cpt_score_maps, cpt_logits, label_logits = part_cem(x)
cpt_score_maps.shape, cpt_logits.shape, label_logits.shape

(torch.Size([8, 113, 7, 7]), torch.Size([8, 112]), torch.Size([8, 200]))

In [ ]:
clip_model, img_transforms = clip.load('RN50', device=device)

In [ ]:
class CLIPModified(nn.Module):
    def __init__(self, model: CLIP):
        super(CLIPModified, self).__init__()
        assert type(model.visual) == ModifiedResNet
        assert type(model.visual.attnpool) == AttentionPool2d

        image_model = copy.deepcopy(model.visual)
        image_model.attnpool = nn.Identity()
        self.model = image_model
        self.attnpool = copy.deepcopy(model.visual.attnpool)

    def forward(self, x: torch.Tensor):
        f = self.model(x) # raw features without attn_pool
        b, c, h, w = f.shape  # [BS, 2048, 7, 7]
        g = self.attnpool(f)  # [BS, 1024]
        x = f.permute(0, 2, 3, 1)  # [BS, 7, 7, 2048]
        x = x.reshape(-1, c)  # [BS*7*7, 2048]
        x = x[..., None, None]  # [BS*7*7, 2048, 1, 1]
        x = x.expand(-1, -1, 7, 7)  # [BS*7*7, 2048, 7, 7]
        x = self.attnpool(x)  # [BS*7*7, 1024]
        x = x.reshape(b, h, w, -1)  # [BS, 7, 7, 1024]
        x = x.permute(0, 3, 1, 2)  # [BS, 1024, 7, 7]
        return f, g, x

clip_modified = CLIPModified(clip_model)

In [ ]:
x = torch.randn(8, 3, 224, 224)
f, g, x = clip_modified(x)
f.shape, g.shape, x.shape

In [ ]:
img_dir = 'datasets/CUB/CUB_200_2011/images/083.White_breasted_Kingfisher/White_Breasted_Kingfisher_0035_73290.jpg'
texts = ['brown bird head', 'black bird head', 'white bird head', 'background']
texts_tokens = clip.tokenize(texts=texts)
img = Image.open(img_dir).convert('RGB')
with torch.no_grad():
    img = img_transforms(img=img)
    features = clip_modified(img.unsqueeze(0))
    texts_encoded = clip_model.encode_text(texts_tokens)
    f, g, x = features

In [ ]:
x, texts_encoded = x.squeeze(0), texts_encoded.squeeze(0)
x = x / x.norm(dim=0, keepdim=True)
texts_encoded = texts_encoded / texts_encoded.norm()
score_map = F.softmax(torch.einsum('kd,dwh->kwh', texts_encoded, x), dim=0)

In [ ]:
score_map.shape

In [ ]:
score_map[0]

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(score_map[0])
plt.show()

In [ ]:
Image.open(img_dir)

In [ ]:
torch.nn.functional.softmax(torch.tensor([-1., -1., 5.0, 5.0]), dim=0)

In [ ]:
scores = torch.randn(32, 100)[..., None]
concepts = torch.rand(100, 1024)[None, ...].expand(32, -1, -1)
(scores * concepts).shape

In [ ]:
ts = concepts.unbind(dim=1)
f = torch.cat(ts, dim=-1)
len(ts), ts[0].shape, f.shape

In [ ]:
102400*100/1000000